In [1]:
# 30 clicks and mode (final with songs)
import tensorflow as tf
model = tf.keras.models.load_model('model_csv.h5')
label_dict = {0 : 'Angry', 1 : 'Disgust', 2 : 'Fear', 3 : 'Happiness', 4 : 'Sad', 5 : 'Surprise', 6 : 'Neutral'}
import cv2
import numpy as np
from keras.preprocessing import image
from pygame import mixer

url = "http://192.168.5.104:8080/video"
face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(url)
count = 0
final_pred = []
while True:
    _,cap_image = cap.read()
    cap_img_gray = cv2.cvtColor(cap_image, cv2.COLOR_BGR2GRAY)
    faces = face_haar_cascade.detectMultiScale(cap_img_gray, 1.3, 5)
    

    for (x,y,w,h) in faces:
        cv2.rectangle(cap_image, (x,y), (x+w,y+h),(255,255,255),2)
        count+=1
        
        #print(count)
        roi_gray = cap_img_gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48,48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis=0)
        predictions = model.predict(img_pixels)
        emotion_label = np.argmax(predictions)
        emotion_prediction = label_dict[emotion_label]
        cv2.putText(cap_image, emotion_prediction, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,200,0), 1 )
        resize_image = cv2.resize(cap_image, (1000,700))
        cv2.imshow('Emotion',resize_image)
        final_pred.append(emotion_prediction)
        
    k = cv2.waitKey(200) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
    elif count >= 30: # Take 30 face sample and stop video
        break
        
#print(final_pred)
test_1 = np.array(final_pred)
from scipy import stats
mode = stats.mode(test_1)
print(mode[0])
cap.release()
cv2.destroyAllWindows()

mixer.init()
if mode[0][0] == 'Neutral':
    mixer.music.load('songs/nutral.mp3')
    mixer.music.play()
elif mode[0][0] == 'Happiness':
    mixer.music.load('songs/happy.mp3')
    mixer.music.play()
elif mode[0][0] == 'Angry':
    mixer.music.load('songs/angry.mp3')
    mixer.music.play()
elif mode[0][0] == 'Sad':
    mixer.music.load('songs/sad.mp3')
    mixer.music.play()
elif mode[0][0] == 'Disgust':
    mixer.music.load('songs/disgust.mp3')
    mixer.music.play()
elif mode[0][0] == 'Fear':
    mixer.music.load('songs/fear.mp3')
    mixer.music.play()
else:
    mixer.music.load('songs/surprised.mp3')
    mixer.music.play()
    

KeyboardInterrupt: 